In [ ]:
### IMPORTS ###

# Built-in Libraries
import numpy as np
import os
import pandas as pd
import random
import re
import string
from string import punctuation

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize #RegexpTokenizer, 
nltk.download("wordnet")
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

In [23]:
### GLOBAL VARIABLES ###

PROJECT_FOLDER = "C://Users/kathleen.trinh/Documents/Nissan/YouTube/"
FILENAME = "Nissan_LEAF_YouTube_ALL"
IN_FILENAME = FILENAME + ".csv"

BADWORDS = [
    'captain', 'character',
    'hawkeye', 'hulk',
    'marvel',
    'player',
]

STOPWORDS = [
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 
    'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 
    'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 
    'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 
    'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 
    'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 
    'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 
    'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 
    'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 
    'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', 
    "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', 
    "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', 
    "won't", 'wouldn', "wouldn't",
    '´', '‘', '’', '“', '”',
    '[deleted]', 'deleted', '[removed]', 'removed',
    'actual', 'actually', 'add', 'ago', 'agree', 'ah', 'also', 'always', 'amazon', 'answer', 'anything', 'anyway', 'anyways',
    'apparently', 'arent', 'as', 'ask', 'aw', 'aww', 'awww',
    'b', 'bad', 'bc', 'believe', 'bit', 'bot', 'brah', 'bro', 'bruh', 'btw',
    'c', 'call', 'cant', 'congrats', 'congratulation', 'congratulations', 'could', 'couldnt',
    'd', 'damn', 'de', 'definitely', 'dont', 'dude', 'didnt', 'duh', 'dunno',
    'e', 'edit', 'either', 'else', 'en', 'etc', 'even', 'ever', 'everyone', 'exactly',
    'f', 'facebook', 'find', 'found', 'ftw', 'fyi',
    'g', 'gave', 'get', 'gets', 'give', 'gives', 'gl', 'go', 'goes', 'golly', 'gon', 'good', 'google', 'gosh', 'got', 'guess', 'guys',
    'h', 'ha', 'hah', 'haha', 'hahaha', 'happen', 'happens', 'happened', 'hasnt', 'heard', 'hello', 'hes', 'hey', 'heya', 'hi',
    'hm', 'hmm', 'hmmm', 'holy', 'homie', 'http', 'https', 'huh',
    'i', 'idfk', 'idk', 'im', 'imo', 'indeed', 'info', 'irdk', 'isnt',
    'j',
    'k', 'kk', 'kkz', 'know',
    'l', 'la', 'lately', 'left', 'let', 'like', 'link', 'lmao', 'lmfao', 'lol', 'look', 'looks', 'lot', 'lots',
    'm', 'make', 'many', 'may', 'maybe', 'meanwhile', 'message', 'mhm', 'mhmm', 'might', 'much',
    'n', 'na', 'nah', 'name', 'nan', 'nearly', 'next', 'no', 'nope', 'now',
    'o', 'oh', 'ok', 'okay', 'omfg', 'omg', 'one', 'op', 'org',
    'p', 'people', 'please', 'post', 'put',
    'q', 'que', 'question', 'quite',
    'r', 'read', 'really', 'reddit', 'regard', 'regards', 'reply', 'repost', 'right', 'rofl',
    's', 'say', 'says', 'said', 'se', 'see', 'seem', 'shes', 'shit', 'shouldnt', 'shucks', 'slightly', 'someone', 'sometime',
    'sometimes', 'sorry', 'still', 'sub', 'sure',
    't', 'ta', 'talk', 'tbh', 'tell', 'thank', 'thanks', 'thanx', 'thing', 'think', 'tho', 'though', 'thought', 'thread', 'thus',
    'tldr', 'told', 'totally', 'truly', 'two', 'ty',
    'u', 'ugh', 'uh', 'um', 'unless', 'upvote', 'upvotes', 'use', 'usual', 'usually',
    'v', 'via',
    'w', 'want', 'wasnt', 'way', 'well', 'werent', 'whoa', 'wiki', 'wikipedia', 'woah', 'word', 'would', 'wouldnt', 'wow', 'wtf',
    'x',
    'y', 'ya', 'yahoo', 'yay', 'yeah', 'yep', 'yes', 'youd', 'youll', 'youre', 'youve', 'yup',
    'z'
]

In [3]:
print(STOPWORDS)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [42]:
### FUNCTIONS ###

def convert_utf8(s):
    return str(s)

def remove_urls(s):
    s = re.sub(r"[^\s]*www.[^\s]*", "", str(s))
    s = re.sub(r"[^\s]*co.uk[^\s]", "", str(s))
    s = re.sub(r"[^\s]*.biz[^\s]", "", str(s))
    s = re.sub(r"[^\s]*.com[^\s]", "", str(s))
    s = re.sub(r"[^\s]*.edu[^\s]", "", str(s))
    s = re.sub(r"[^\s]*.gov[^\s]", "", str(s))
    s = re.sub(r"[^\s]*.info[^\s]", "", str(s))
    s = re.sub(r"[^\s]*.net[^\s]", "", str(s))
    s = re.sub(r"[^\s]*.org[^\s]", "", str(s))
    return s

def remove_mentions(s):
    s = re.sub(r"\@\b\w*\s", "", s)
    return s
    
def remove_star_words(s):
    return re.sub(r"[^\s]*[\*]+[^\s]*", "", str(s))

def remove_numbers(s):
    return re.sub(r"[^\s]*[0-9]+[^\s]*", "", str(s))

def remove_punctuation(s):
    global punctuation
    for p in punctuation:
        s = str(s).replace(p, " ")
    return s

def remove_shortwords(s):
    s = word_tokenize(s)
    s = " ".join([w for w in s if len(w) > 3])
    return s

# Using default global en_stopwords as list of stopwords
def remove_stopwords(s):
    global en_stopwords
    s = word_tokenize(s)
    s = " ".join([w for w in s if w not in en_stopwords])
    return s

# Using user-defined STOPWORDS as list of stopwords
def remove_stopwords(s):
    s = word_tokenize(s)
    no_stopwords = []
    for w in s:
        if w not in STOPWORDS:
            no_stopwords.append(w)
    no_stopwords = " ".join(no_stopwords)
    return no_stopwords

# NOT USED
def get_lemma_v1(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

# NOT USED
def get_lemma_v2(word):
    return WordNetLemmatizer().lemmatize(word)

def penn2morphy(penntag, returnNone=False):
    morphy_tag = {'NN':wn.NOUN, 'JJ':wn.ADJ,
                  'VB':wn.VERB, 'RB':wn.ADV}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return None if returnNone else ''

def lemmatize(s):
    lemmas = []
    tokens = word_tokenize(s)
    tagged_tokens = nltk.pos_tag(tokens)
    for pair in tagged_tokens:
        new_pos = penn2morphy(pair[1])
        if new_pos != "":
            lemma = WordNetLemmatizer().lemmatize(pair[0], pos = new_pos)
        else:
            lemma = WordNetLemmatizer().lemmatize(pair[0])
        lemmas.append(lemma)
    s = " ".join(lemmas)
    return s

def drop_empty_rows(df):
    for index, row in df.iterrows():
        if row["clean_body_no_stopwords"] == "":
            df.drop(index, inplace=True)
        elif row["clean_body_no_stopwords"] == "NaN":
            df.drop(index, inplace=True)
        elif row["clean_body"] == "deleted":
            df.drop(index, inplace=True)
        elif row["clean_body"] == "removed":
            df.drop(index, inplace=True)
        elif row["body"] == "body":
            df.drop(index, inplace=True)
    return df

In [33]:
### SET-UP ###

# Load .csv file and put data in dataframe
df = pd.read_csv((PROJECT_FOLDER + IN_FILENAME), header=0, usecols=["my_comment_id", "comment"], encoding='utf-8')

#df.columns = ["My Comment ID", "Body"]
df.columns = ["my_comment_id", "body"]

# Print the dataframe's dimentsions
print("Dataframe Dimensions: {} Rows, {} Columns".format(*df.shape))

# Print a sample of 5 elements from the dataframe
df.sample(5)

Dataframe Dimensions: 25434 Rows, 2 Columns


,my_comment_id,body
21852,YT_ZH7V2tU3iFc_Com3417,So Matt let a lady push his Tesla while he was...
17140,YT_F4yXYyzVUjo_Com280,"Blake Morgan \r\nYeah, dumbest ducking comment..."
23524,YT_ZH7V2tU3iFc_Com5089,@TheDanishSpaceman efficiency on a test of M...
12546,YT_bnIg-aEHD-Y_Com1045,This was a brillient review. Very through. Lot...
10410,YT_deXM0MQ2t30_Com10,"Just checked the mileage on my Leaf again, now..."


In [34]:
### TEST FUNCTIONS WITH SUBSET ###

test_df = df.sample(25)

In [43]:
### TEXT PRE-PROCESSING PART 1: CHAR REMOVAL & CONVERSION ###

# Add a new column, "Clean Body", to the training dataframe
# by converting each document's "Body" to type string
test_df["clean_body"] = test_df["body"].map(convert_utf8)

# Remove mentions to other usernames from each document's "Clean Body"
test_df["clean_body"] = test_df["clean_body"].map(remove_mentions)

# Remove URLs from each document's "Clean Body"
test_df["clean_body"] = test_df["clean_body"].map(remove_urls)

# Remove star words from each document's "Clean Body"
test_df["clean_body"] = test_df["clean_body"].map(remove_star_words)

# Remove numbers from each document's "Clean Body"
test_df["clean_body"] = test_df["clean_body"].map(remove_numbers)

# Remove punctuation marks from each document's "Clean Body"
test_df["clean_body"] = test_df["clean_body"].map(remove_punctuation)

# Convert all characters in each document's "Clean Body" to lowercase
test_df["clean_body"] = test_df["clean_body"].map(lambda x: x.lower())

# Print a sample of 5 elements from the training dataframe
test_df.sample(5)

,my_comment_id,body,clean_body
3262,YT_UMKDqIzpQGA_Com174,"Hi, I have 2013 Nissan leaf. My battery having...",hi i have nissan leaf my battery having ba...
16334,YT_dZRx2fKs70c_Com150,It’s a built in sat Nav that’s not functioning...,it’s a built in sat nav that’s not functioning...
19928,YT_ZH7V2tU3iFc_Com1493,@Han Solo Yes not even close ...,solo yes not even close
3255,YT_UMKDqIzpQGA_Com167,Have you checked your SOH recently?,have you checked your soh recently
6770,YT_7HJWz58O9ac_Com118,Looks like Nissan just wants to keep the LEAF ...,looks like nissan just wants to keep the leaf ...


In [44]:
### TEXT PRE-PROCESSING PART 2: STOPWORDS ###

# Add a new column, "Clean Body no stopwords", to the training dataframe,
# which is the text from "Clean Body" with NO stopwords
test_df["clean_body_no_stopwords"] = test_df["clean_body"].map(remove_stopwords)

# Remove short words (3 chars or less)
#train_df["Clean Body no stopwords"] = train_df["Clean Body"].map(remove_shortwords)

# Print a sample of 5 elements from the training dataframe
test_df.sample(5)

,my_comment_id,body,clean_body,clean_body_no_stopwords
18658,YT_ZH7V2tU3iFc_Com223,@evil701 - A lot of countries don't have cle...,a lot of countries don t have clean drink...,countries clean drinking water proper toilets ...
15386,YT_trAJaqiT3Wc_Com397,I bought a Leaf Tekna in March 2014 with gover...,i bought a leaf tekna in march rnment discoun...,bought leaf tekna march rnment discount brothe...
13481,YT_W1OpW5XDliw_Com141,We're exactly the same always trying to justif...,we re exactly the same always trying to justif...,trying justify going electric car kids disappo...
23348,YT_ZH7V2tU3iFc_Com4913,Love how you say wish me luck,love how you say wish me luck,love wish luck
3077,YT__2FyYLroby4_Com117,"It’s a light hearted comment that’s all, you s...",it’s a light ent that’s all you should be hap...,light ent happy watched given crass click bait...


In [45]:
test_df

,my_comment_id,body,clean_body,clean_body_no_stopwords
13481,YT_W1OpW5XDliw_Com141,We're exactly the same always trying to justif...,we re exactly the same always trying to justif...,trying justify going electric car kids disappo...
4249,YT__SEs9I9sptw_Com48,"I recently discovered this channel, and I love...",i recently discovered this channel and i love...,recently discovered channel love mation real c...
3255,YT_UMKDqIzpQGA_Com167,Have you checked your SOH recently?,have you checked your soh recently,checked soh recently
15386,YT_trAJaqiT3Wc_Com397,I bought a Leaf Tekna in March 2014 with gover...,i bought a leaf tekna in march rnment discoun...,bought leaf tekna march rnment discount brothe...
3262,YT_UMKDqIzpQGA_Com174,"Hi, I have 2013 Nissan leaf. My battery having...",hi i have nissan leaf my battery having ba...,nissan leaf battery bar replace new battery kw...
3077,YT__2FyYLroby4_Com117,"It’s a light hearted comment that’s all, you s...",it’s a light ent that’s all you should be hap...,light ent happy watched given crass click bait...
7311,YT_C4nS_tSQiVQ_Com382,"Yes, it’s planned. Most of them haven’t had mu...",yes it’s planned most of them haven’t had mu...,planned physically show us yet
23348,YT_ZH7V2tU3iFc_Com4913,Love how you say wish me luck,love how you say wish me luck,love wish luck
6850,YT_7HJWz58O9ac_Com198,To add to my prev comment we took that same ca...,to add to my ent we took that same car from po...,ent took car poole bristol yesterday drove mil...
7368,YT_C4nS_tSQiVQ_Com439,more accurate but less catchy title: Upgrading...,more accurate but less catchy title upgrading...,accurate less catchy title upgrading gen nissa...


In [46]:
### CLEAN ENTIRE CORPUS IF TEST RESULTS ARE OK ###

In [47]:
### TEXT PRE-PROCESSING PART 1: CHAR REMOVAL & CONVERSION ###

# Add a new column, "Clean Body", to the training dataframe
# by converting each document's "Body" to type string
df["clean_body"] = df["body"].map(convert_utf8)

# Remove URLs from each document's "Clean Body"
df["clean_body"] = df["clean_body"].map(remove_urls)

# Remove star words from each document's "Clean Body"
df["clean_body"] = df["clean_body"].map(remove_star_words)

# Remove numbers from each document's "Clean Body"
df["clean_body"] = df["clean_body"].map(remove_numbers)

# Remove punctuation marks from each document's "Clean Body"
df["clean_body"] = df["clean_body"].map(remove_punctuation)

# Convert all characters in each document's "Clean Body" to lowercase
df["clean_body"] = df["clean_body"].map(lambda x: x.lower())

# Print a sample of 5 elements from the training dataframe
df.sample(5)

,my_comment_id,body,clean_body
23906,YT_ZH7V2tU3iFc_Com5471,“Sponsored advert”; are you tripping? 😂,“sponsored advert” are you tripping 😂
15555,YT_tMElcgSwxSU_Com126,1950s: There will be beautiful cars in the fut...,there will be beautiful cars in the future\r\...
8574,YT_DqTA_gA7exs_Com204,Typical calculations of CO2 per km for a gasol...,typical calculations of per km for a gasoline...
6507,YT_SVnmdJy0qc8_Com310,"James Hamilton if you want seats for effiency,...",james hamilton if you want seats for effiency ...
14321,YT_8dljcs42R-0_Com11,my partner has the same look when we go chargi...,my partner has the same look when we go chargi...


In [48]:
### TEXT PRE-PROCESSING PART 2: STOPWORDS ###

# Add a new column, "Clean Body no stopwords", to the training dataframe,
# which is the text from "Clean Body" with NO stopwords
df["clean_body_no_stopwords"] = df["clean_body"].map(remove_stopwords)

# Remove short words (3 chars or less)
#train_df["Clean Body no stopwords"] = train_df["Clean Body"].map(remove_shortwords)

In [49]:
# Print a sample of 5 elements from the training dataframe
df.sample(5)

,my_comment_id,body,clean_body,clean_body_no_stopwords
20112,YT_ZH7V2tU3iFc_Com1677,The Leaf did better than I expected. I'd rathe...,the leaf did better than i expected i d rathe...,leaf better expected rather kia discounts leaf...
5467,YT_I6ki0WuYUB0_Com497,"This is supposed to be their most appealing, f...",this is supposed to be their most appealing f...,supposed appealing futuristic car encourage ma...
11087,YT_aWcf932twbM_Com50,"With the 60kWh battery, rapid charging shouldn...",with the battery rapid charging shouldn t be...,battery rapid charging issue miles per charge ...
17725,YT_bMJKh_wQ-W0_Com569,Dacia Spring??!,dacia spring,dacia spring
6380,YT_SVnmdJy0qc8_Com183,"6 months PLUS the delivery period, which could...",months plus the delivery period which could ...,months plus delivery period mean another months


In [50]:
### TEXT PRE-PROCESSING PART 3: LEMMATIZATION ###

# Lemmatize each token in each document
df["clean_body_no_stopwords"] = df["clean_body_no_stopwords"].map(lemmatize)

In [51]:
# Print a sample of 5 elements from the training dataframe
df.sample(5)

,my_comment_id,body,clean_body,clean_body_no_stopwords
23812,YT_ZH7V2tU3iFc_Com5377,Not even close to a funny joke,not even close to a funny joke,close funny joke
5471,YT_I6ki0WuYUB0_Com501,"Since Renault took control, Nissan have lost t...",since renault took control nissan have lost t...,since renault take control nissan lose plot mi...
21491,YT_ZH7V2tU3iFc_Com3056,"@George L not really, because for the Kia th...",l not really because for the kia they used...,kia use estimate ute base temperature tesla us...
18943,YT_ZH7V2tU3iFc_Com508,Money,money,money
16792,YT_Ldip8KhHhVY_Com48,"Rimas Bestauskas, I didn't like to rub it in ...",rimas bestauskas i didn t like to rub it in ...,rima bestauskas rub leaf owner range satnav me...


In [52]:
### TEXT PRE-PROCESSING PART 4: STOPWORDS (AGAIN) ###

# Add a new column, "Clean Body no stopwords", to the training dataframe,
# which is the text from "Clean Body" with NO stopwords
df["clean_body_no_stopwords"] = df["clean_body_no_stopwords"].map(remove_stopwords)

# Remove short words (3 chars or less)
#train_df["Clean Body no stopwords"] = train_df["Clean Body"].map(remove_shortwords)

In [53]:
# Print a sample of 5 elements from the training dataframe
df.sample(5)

,my_comment_id,body,clean_body,clean_body_no_stopwords
6781,YT_7HJWz58O9ac_Com129,@Richard Petek Charging to 90% overnight at ...,richard petek charging to overnight at hom...,richard petek charge overnight home road trip ...
14822,YT_NE0D7RAOecE_Com152,I would keep the car.,i would keep the car,keep car
7552,YT_C4nS_tSQiVQ_Com623,I wonder if the Brexit mess is affecting the t...,i wonder if the brexit mess is affecting the t...,wonder brexit mess affect trade europe batt on...
17335,YT_bMJKh_wQ-W0_Com179,For the BMW i3 it wasn;t that it looked differ...,for the bmw it wasn t that it looked differen...,bmw different cool expensive
24295,YT_ZH7V2tU3iFc_Com5860,4:57 Bearded Man in the Audi. 4:57 No more Bea...,bearded man in the audi no more bearded man...,beard man audi beard man ooo


In [54]:
### DROP EMPTY ROWS ###

# Print the dataframe's dimensions
print("Dataframe Dimensions (with empty rows): {} Rows, {} Columns".format(*df.shape))

try:
    df = drop_empty_rows(df)
except MemoryError as e:
    print("Memory Error thrown:")
    print(e)

Dataframe Dimensions (with empty rows): 25434 Rows, 4 Columns


In [55]:
# Print the dataframe's dimensions
print("Dataframe Dimensions (after removing empty rows): {} Rows, {} Columns".format(*df.shape))

Dataframe Dimensions (after removing empty rows): 25041 Rows, 4 Columns


In [56]:
df

,my_comment_id,body,clean_body,clean_body_no_stopwords
0,YT_0dxaQ0xAS9c_Com1,The heated seats do not use much electricity a...,the heated seats do not use much electricity a...,heat seat electricity none less absolutely but...
1,YT_0dxaQ0xAS9c_Com2,Your confident delivery and thoughtful observa...,your confident delivery and thoughtful observa...,confident delivery thoughtful observation long...
2,YT_0dxaQ0xAS9c_Com3,Thank you so much! I've certainly grown ALOT i...,thank you so much i ve certainly grown alot i...,certainly grown alot confidence
3,YT_0dxaQ0xAS9c_Com4,Got a new Leaf and so far love it. Happy to se...,got a new leaf and so far love it happy to se...,new leaf far love happy eting leaf town
4,YT_0dxaQ0xAS9c_Com5,Really glad you're enjoying your new Leaf!,really glad you re enjoying your new leaf,glad enjoy new leaf
...,...,...,...,...
25429,YT_ZH7V2tU3iFc_Com6994,@medler2110 go for it. But other then the ra...,go for it but other then the range it can...,range ared
25430,YT_ZH7V2tU3iFc_Com6995,"Maybe they are, but with £15k I could do plent...",maybe they are but with i could do plenty of...,plenty relieve disappointment tesla kia work h...
25431,YT_ZH7V2tU3iFc_Com6996,@medler2110 Absolutely fine. Buy any EV you ...,absolutely fine buy any ev you like,absolutely fine buy ev
25432,YT_ZH7V2tU3iFc_Com6997,Tesla is the best,tesla is the best,tesla best


In [57]:
### OUTPUT ###

df.to_csv((PROJECT_FOLDER + FILENAME + "_CLEAN.csv"), index=None, header=True, encoding='utf-8')

In [58]:
### MAKE SHUFFLED DATAFRAME ###

shuffled_df = df.sample(frac=1)

In [59]:
shuffled_df

,my_comment_id,body,clean_body,clean_body_no_stopwords
6508,YT_SVnmdJy0qc8_Com311,TrueDesireHD . I like leather seats. Mmmm.....,truedesirehd i like leather seats mmmm,truedesirehd leather seat mmmm
21653,YT_ZH7V2tU3iFc_Com3218,4250 employees of Shell Offshore Oil explorati...,employees of shell offshore oil exploration d...,employee shell offshore oil exploration dislik...
19533,YT_ZH7V2tU3iFc_Com1098,@milosilic23 what? Aren't they? In Munich I'...,what aren t they in munich i ve seen some...,munich charge station street bmws volvos audis...
5900,YT_I6ki0WuYUB0_Com930,Its not French.,its not french,french
23712,YT_ZH7V2tU3iFc_Com5277,And in winter weather when the car is 4 years ...,and in winter weather when the car is years o...,winter weather car year old expect anywhere ne...
...,...,...,...,...
21372,YT_ZH7V2tU3iFc_Com2937,@CageyBee yeah Tesla batters don't cost 25k ...,cageybee yeah tesla batters don t cost to ...,cageybee tesla batter cost replace try rat lif...
490,YT_2qPF11PmP8k_Com437,IIRC the deal was when his car stopped his loc...,iirc the deal was when his car stopped his loc...,iirc deal car stop local garage collect lap ar...
3079,YT__2FyYLroby4_Com119,"@Modern Heroes Good point, hard to get peopl...",modern heroes good point hard to get peopl...,modern hero point hard watch content regardles...
16962,YT_F4yXYyzVUjo_Com102,"Leaf higher power,range,space and lower cost w...",leaf higher power range space and lower cost w...,leaf high power range space low cost whereas v...


In [33]:
### OUTPUT SHUFFLED DATAFRAME ###

df.to_csv((PROJECT_FOLDER + FILENAME + "_CLEAN_SHUFFLED.csv"), index=None, header=True, encoding='utf-8')